#**Graph**

#**Installations and Imports**

In [17]:
import time

import random

import math

import numpy as np

import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

#**Graph Representation**

**Node**

In [3]:
class Node:
    """
    Class representing a node in a graph.

    Attributes:
    ----------
    id : A unique identifier for the node.
    data : The data or value that node holds.
    visited : Bool indicating if the node has been visited in a traversal.
    initial_visit_order : Optional visit order in a traversal.
    finishing_visit_order : Optional finishing visit order in a traversal.
    edges : A dict holding IDs of nodes that have an edge from this node, and edge weight.
    """

    def __init__(self, id, data=None, visited=False,
                 initial_visit_order=None, finishing_visit_order=None):
        self.id = id
        self.data = data
        self.visited = visited
        self.initial_visit_order = initial_visit_order
        self.finishing_visit_order = finishing_visit_order
        self.edges = {}

    def __repr__(self):
        edges_str = ", ".join(f"{k}: {v}" for k, v in self.edges.items())
        return f"<Node(id={self.id}, data={self.data}, edges={{{edges_str}}})>"

**Adjacency List**

In [4]:
class AdjacencyList:
    """
    Class representing a Graph as an Adjacency List.

    Attributes:
        nodes (dict): A dictionary mapping node identifiers to Node objects.
        directed (bool): A flag indicating whether the graph is directed or not.
    """

    def __init__(self, directed=True):
        """
        Initialize a graph.

        Parameters:
            directed (bool): If True, the graph is directed. If False, the graph is undirected. Defaults to True.
        """
        self.nodes = {}
        self.directed = directed

    def add_node(self, id, data=None):
        """
        Add a node to the graph.

        Args:
            id : The identifier for the node.
            data (optional): The data to be stored in the node. Defaults to None.
        """
        if id not in self.nodes:
            self.nodes[id] = Node(id, data)

    def remove_node(self, id):
        """
        Remove a node from the graph if it exists.

        Args:
            id : The identifier of the node.

        Raises:
            KeyError: If the node does not exist.
        """
        if id in self.nodes:
            node = self.nodes[id]
            for neighbor_id in list(node.edges.keys()):
                self.remove_edge(id, neighbor_id, check_existence=False)
            del self.nodes[id]
        else:
            raise KeyError(f"Node with id {id} does not exist.")

    def add_edge(self, source, destination, weight=None):
        """
        Add an edge between existing nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.
            weight (optional): The weight of the edge. Defaults to None.

        Raises:
            ValueError: If the source or destination node does not exist.
        """
        if source in self.nodes and destination in self.nodes:
            self.nodes[source].edges[destination] = weight
            if not self.directed:
                self.nodes[destination].edges[source] = weight
        else:
            raise ValueError("Both source and destination nodes must exist.")

    def remove_edge(self, source, destination, check_existence=True):
        """
        Remove an edge between existing nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.
            check_existence : If False, skip the existence check. Useful when calling from 'remove_node'.

        Raises:
            KeyError: If the source or destination node does not exist.
        """
        if not check_existence or (source in self.nodes and destination in self.nodes):
            del self.nodes[source].edges[destination]
            if not self.directed:
                del self.nodes[destination].edges[source]
        else:
            raise KeyError("Both source and destination nodes must exist.")

    def edge_exists(self, source, destination):
        """
        Check if an edge exists between specified nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.

        Returns:
            bool: True if an edge exists between the specified nodes, False otherwise.
        """
        return source in self.nodes and destination in self.nodes[source].edges

    def get_edge_weight(self, source, destination):
        """
        Get the weight of an edge between specified nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.

        Returns:
            weight: The weight of the edge, or None if the edge does not exist.
        """
        if self.edge_exists(source, destination):
            return self.nodes[source].edges[destination]

    def get_node_data(self, id):
        """
        Get the data associated with a node.

        Args:
            id : The identifier of the node.

        Returns:
            data: The data associated with the node, or None if the node does not exist.
        """
        if id in self.nodes:
            return self.nodes[id].data

    def set_node_data(self, id, data):
        """
        Set the data for a node.

        Args:
            id : The identifier of the node.
            data : The new data for the node.
        """
        if id in self.nodes:
            self.nodes[id].data = data

    def get_nodes(self):
        """
        Get all node ids in the graph.

        Returns:
            List[int or str]: A list of all node ids in the graph.
        """
        return list(self.nodes.keys())

    def get_edges(self):
        """
        Get all edges in the graph.

        Returns:
            List[Tuple]: A list of all edges in the graph, represented as tuples of (source, destination, weight).
        """
        edges = []
        for id, node in self.nodes.items():
            for neighbor, weight in node.edges.items():
                # If the graph is undirected and the edge hasn't been added yet, add it
                if not self.directed and (neighbor, id, weight) not in edges:
                    edges.append((id, neighbor, weight))
                # If the graph is directed, add the edge
                elif self.directed:
                    edges.append((id, neighbor, weight))
        return edges

    def clear(self):
        """
        Clear all nodes and edges from the graph.
        """
        self.nodes.clear()

    def size(self):
        """
        Get the number of nodes in the graph.

        Returns:
            int: The number of nodes in the graph.
        """
        return len(self.nodes)

    def is_empty(self):
        """
        Check if the graph is empty.

        Returns:
            bool: True if the graph has no nodes, False otherwise.
        """
        return len(self.nodes) == 0

    def neighbors(self, id):
        """
        Get the neighbors of a node.

        Args:
            id : The identifier of the node.

        Returns:
            List[int or str]: A list of identifiers for the neighbors of the node.
        """
        if id in self.nodes:
            return list(self.nodes[id].edges.keys())

    def has_node(self, id):
        """
        Check if a node exists in the graph.

        Args:
            id : The identifier of the node.

        Returns:
            bool: True if the node exists in the graph, False otherwise.
        """
        return id in self.nodes

    def has_edge(self, source, destination):
        """
        Check if an edge exists between specified nodes.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.

        Returns:
            bool: True if an edge exists between the specified nodes, False otherwise.
        """
        return self.edge_exists(source, destination)

    def __repr__(self):
        """
        Get a string representation of the graph.

        Returns:
            str: A string representation of the graph.
        """
        return "\n".join(str(node) for node in self.nodes.values())

In [5]:
# Create an instance of AdjacencyList
graph = AdjacencyList(directed=False)

print("\n--> Initial State of the Graph")
print("Nodes:", graph.get_nodes())  # At this stage, the graph should be empty

# Adding nodes
for i in range(4):
    graph.add_node(i, f"node{i}")

print("\n--> After Adding Nodes")
print("Nodes:", graph.get_nodes())  # The graph should now have four nodes with respective data

# Adding edges with weights
edges_to_add = [(0, 1, 3), (0, 2, 2), (1, 3, 1), (2, 3, 4)]
for edge in edges_to_add:
    source, dest, weight = edge
    graph.add_edge(source, dest, weight)

print("\n--> After Adding Edges with Weights")
print("Edges:", graph.get_edges())  # The graph should now have edges with weights

# Check if certain edges exist
print("\n--> Edge Existence Check")
edges_to_check = [(0, 1), (1, 0)]
for edge in edges_to_check:
    source, dest = edge
    print(f"Edge between {source} and {dest}: {graph.edge_exists(source, dest)}")

# Getting Edge Weight
print("\n--> Getting Edge Weight")
edges_to_check_weight = [(0, 1)]
for edge in edges_to_check_weight:
    source, dest = edge
    print(f"Edge weight between {source} and {dest}: {graph.get_edge_weight(source, dest)}")

# Setting and Getting Node Data
graph.set_node_data(0, "updatedNode0")
print("\n--> Getting Node Data")
print(f"Data of node 0: {graph.get_node_data(0)}")  # This should return 'updatedNode0'

# Getting Nodes and Edges
print("\n--> Getting Nodes and Edges")
print("All Nodes:", graph.get_nodes())
print("All Edges:", graph.get_edges())

# Checking Neighbors of a Node
node_to_check_neighbors = 0
print(f"\n--> Checking Neighbors of Node {node_to_check_neighbors}")
print(f"Neighbors of Node {node_to_check_neighbors}:", graph.neighbors(node_to_check_neighbors))

# Checking Graph Properties
print("\n--> Checking Graph Properties")
print(f"Graph Size:", graph.size())
print(f"Is Graph Empty:", graph.is_empty())

# Removing some edges
edges_to_remove = [(0, 1), (2, 3)]
for edge in edges_to_remove:
    source, dest = edge
    graph.remove_edge(source, dest)

print("\n--> After Removing Some Edges")
print("Edges:", graph.get_edges())  # The graph should now have fewer edges

# Remove a node
nodes_to_remove = [1]
for node in nodes_to_remove:
    graph.remove_node(node)

print("\n--> After Removing Nodes")
print("Nodes:", graph.get_nodes())  # The graph should now have fewer nodes and corresponding edges

# Clearing the Graph
graph.clear()

print("\n--> After Clearing the Graph")
print("Nodes:", graph.get_nodes())  # At this stage, the graph should be empty again

# Adding nodes and edges again to check the operations after clearing
graph.add_node(4, "node4")
graph.add_node(5, "node5")
graph.add_edge(4, 5, 6)  # Adding edge between node4 and node5 with weight 6

print("\n--> After Adding Nodes and Edge After Clearing")
print("Nodes:", graph.get_nodes())  # The graph should now have two nodes
print("Edges:", graph.get_edges())  # The graph should now have one edge

# Check the data of a node
node_to_check_data = 4
print(f"\n--> Checking Node Data for node {node_to_check_data}")
print(f"Data of node {node_to_check_data}:", graph.get_node_data(node_to_check_data))  # This should return 'node4'

# Check the weight of an edge
edge_to_check_weight = (4, 5)
print(f"\n--> Checking Edge Weight for edge {edge_to_check_weight}")
print(f"Edge weight between {edge_to_check_weight[0]} and {edge_to_check_weight[1]}: {graph.get_edge_weight(*edge_to_check_weight)}")  # This should return 6

# Remove a node
nodes_to_remove = [4]
for node in nodes_to_remove:
    graph.remove_node(node)

print("\n--> After Removing Nodes")
print("Nodes:", graph.get_nodes())  # The graph should now have one node

# Check if the graph is empty
print("\n--> Checking If Graph Is Empty")
print("Is Graph Empty:", graph.is_empty())  # This should return False



--> Initial State of the Graph
Nodes: []

--> After Adding Nodes
Nodes: [0, 1, 2, 3]

--> After Adding Edges with Weights
Edges: [(0, 1, 3), (0, 2, 2), (1, 3, 1), (2, 3, 4)]

--> Edge Existence Check
Edge between 0 and 1: True
Edge between 1 and 0: True

--> Getting Edge Weight
Edge weight between 0 and 1: 3

--> Getting Node Data
Data of node 0: updatedNode0

--> Getting Nodes and Edges
All Nodes: [0, 1, 2, 3]
All Edges: [(0, 1, 3), (0, 2, 2), (1, 3, 1), (2, 3, 4)]

--> Checking Neighbors of Node 0
Neighbors of Node 0: [1, 2]

--> Checking Graph Properties
Graph Size: 4
Is Graph Empty: False

--> After Removing Some Edges
Edges: [(0, 2, 2), (1, 3, 1)]

--> After Removing Nodes
Nodes: [0, 2, 3]

--> After Clearing the Graph
Nodes: []

--> After Adding Nodes and Edge After Clearing
Nodes: [4, 5]
Edges: [(4, 5, 6)]

--> Checking Node Data for node 4
Data of node 4: node4

--> Checking Edge Weight for edge (4, 5)
Edge weight between 4 and 5: 6

--> After Removing Nodes
Nodes: [5]

--> Ch

**Adjacency Matrix**

In [6]:
class AdjacencyMatrix:
    """
    Class representing a Graph as an Adjacency Matrix.

    Attributes:
        matrix (dict): A dictionary representing the adjacency matrix.
        directed (bool): A flag indicating whether the graph is directed or not.
        node_data (dict): A dictionary mapping node identifiers to data.
    """

    def __init__(self, directed=True):
        """
        Initialize a graph.

        Parameters:
            directed (bool): If True, the graph is directed. If False, the graph is undirected. Defaults to True.
        """
        self.matrix = {}
        self.directed = directed
        self.node_data = {}

    def add_node(self, id, data=None):
        """
        Add a node to the graph.

        Args:
            id : The identifier for the node.
            data : The data to be stored in the node.
        """
        if id not in self.matrix:
            self.matrix[id] = {}
            self.node_data[id] = data

    def remove_node(self, id):
        """
        Remove a node from the graph if it exists.

        Args:
            id : The identifier of the node.

        Raises:
            ValueError: If the node does not exist.
        """
        if id in self.matrix:
            del self.matrix[id]
            for node in self.matrix:
                if id in self.matrix[node]:
                    del self.matrix[node][id]
            del self.node_data[id]
        else:
            raise KeyError(f"Node with id {id} does not exist.")

    def add_edge(self, source, destination, weight=1):
        """
        Add an edge between existing nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.
            weight : The weight of the edge.
        """
        if source in self.matrix and destination in self.matrix:
            self.matrix[source][destination] = weight
            if not self.directed:
                self.matrix[destination][source] = weight
        else:
            raise ValueError("Both source and destination nodes must exist.")

    def remove_edge(self, source, destination):
        """
        Remove an edge between existing nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.
        """
        if source in self.matrix and destination in self.matrix:
            del self.matrix[source][destination]
            if not self.directed and source in self.matrix[destination]:
                del self.matrix[destination][source]

    def get_edge_weight(self, source, destination):
        """
        Get the weight of an edge between specified nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.

        Returns:
            weight: The weight of the edge, or None if the edge does not exist.
        """
        return self.matrix[source][destination] if self.edge_exists(source, destination) else None

    def get_node_data(self, id):
        """
        Get the data associated with a node.

        Args:
            id : The identifier of the node.

        Returns:
            data: The data associated with the node, or None if the node does not exist.
        """
        return self.node_data.get(id)

    def set_node_data(self, id, data):
        """
        Set the data for a node.

        Args:
            id : The identifier of the node.
            data : The new data for the node.
        """
        if id in self.matrix:
            self.node_data[id] = data

    def edge_exists(self, source, destination):
        """
        Check if an edge exists between specified nodes in the graph.

        Args:
            source : The identifier of the source node.
            destination : The identifier of the destination node.

        Returns:
            bool: True if an edge exists between the specified nodes, False otherwise.
        """
        return source in self.matrix and destination in self.matrix[source]

    def has_node(self, id):
        """
        Check if a node exists in the graph.

        Args:
            id : The identifier of the node.

        Returns:
            bool: True if the node exists in the graph, False otherwise.
        """
        return id in self.matrix

    def neighbors(self, id):
        """
        Get the neighbors of a node.

        Args:
            id : The identifier of the node.

        Returns:
            List[int or str]: A list of identifiers for the neighbors of the node.
        """
        return list(self.matrix[id].keys()) if id in self.matrix else []

    def is_empty(self):
        """
        Check if the graph is empty.

        Returns:
            bool: True if the graph has no nodes, False otherwise.
        """
        return len(self.matrix) == 0

    def size(self):
        """
        Get the number of nodes in the graph.

        Returns:
            int: The number of nodes in the graph.
        """
        return len(self.matrix)

    def clear(self):
        """
        Clear all nodes and edges from the graph.
        """
        self.matrix.clear()
        self.node_data.clear()

    def get_nodes(self):
        """
        Get all node ids in the graph.

        Returns:
            List[int or str]: A list of all node ids in the graph.
        """
        return list(self.matrix.keys())

    def get_edges(self):
        """
        Get all edges in the graph.

        Returns:
            List[Tuple]: A list of all edges in the graph, represented as tuples of (source, destination, weight).
        """
        edges = []
        for id, neighbors in self.matrix.items():
            for neighbor, weight in neighbors.items():
                edges.append((id, neighbor, weight))
        return edges

    def __repr__(self):
        """
        Get a string representation of the graph.

        Returns:
            str: A string representation of the graph.
        """
        return '\n'.join([f"Node {node}: data -> {self.node_data[node]}, edges -> {neighbors}" for node, neighbors in self.matrix.items()])


In [7]:
# Create an instance of AdjacencyMatrix
graph = AdjacencyMatrix(directed=False)

print("\n---> Initial State of the Graph")
print(f"All Nodes: {graph.get_nodes()}")  # At this stage, the graph should be empty

# Adding nodes
graph.add_node(0, "node0")
graph.add_node(1, "node1")
graph.add_node(2, "node2")
graph.add_node(3, "node3")

print("\n---> After Adding Nodes")
print(f"All Nodes: {graph.get_nodes()}")  # The graph should now have four nodes with respective data

# Adding edges with weights
graph.add_edge(0, 1, 3)  # Adding edge between node0 and node1 with weight 3
graph.add_edge(0, 2, 2)  # Adding edge between node0 and node2 with weight 2
graph.add_edge(1, 3, 1)  # Adding edge between node1 and node3 with weight 1
graph.add_edge(2, 3, 4)  # Adding edge between node2 and node3 with weight 4

print("\n---> After Adding Edges with Weights")
print(f"All Edges: {graph.get_edges()}")  # The graph should now have edges with weights

# Check if certain edges exist
print("\n---> Edge Existence Check")
print(f"Edge between 0 and 1: {graph.edge_exists(0, 1)}")  # This should return True
print(f"Edge between 1 and 0: {graph.edge_exists(1, 0)}")  # This should return True (since our graph is undirected)

# Getting Edge Weight
print("\n---> Getting Edge Weight")
print(f"Edge weight between 0 and 1: {graph.get_edge_weight(0, 1)}")  # This should return 3

# Setting and Getting Node Data
graph.set_node_data(0, "updatedNode0")
print("\n---> Getting Node Data")
print(f"Data of node 0: {graph.get_node_data(0)}")  # This should return 'updatedNode0'

# Checking Neighbors of a Node
print("\n---> Checking Neighbors of Node 0")
print(f"Neighbors of Node 0: {graph.neighbors(0)}")

# Checking Graph Properties
print("\n---> Checking Graph Properties")
print(f"Graph Size: {graph.size()}")
print(f"Is Graph Empty: {graph.is_empty()}")

# Removing some edges
graph.remove_edge(0, 1)  # Removing edge between node0 and node1
graph.remove_edge(2, 3)  # Removing edge between node2 and node3

print("\n---> After Removing Some Edges")
print(f"All Edges: {graph.get_edges()}")  # The graph should now have fewer edges

# Remove a node
graph.remove_node(1)  # Removing node1

print("\n---> After Removing Node 1")
print(f"All Nodes: {graph.get_nodes()}")  # The graph should now have fewer nodes and corresponding edges

# Clearing the Graph
graph.clear()

print("\n---> After Clearing the Graph")
print(f"All Nodes: {graph.get_nodes()}")  # At this stage, the graph should be empty again

# Adding nodes and edges again to check the operations after clearing
graph.add_node(4, "node4")
graph.add_node(5, "node5")
graph.add_edge(4, 5, 6)  # Adding edge between node4 and node5 with weight 6

print("\n---> After Adding Nodes and Edge After Clearing")
print(f"All Nodes: {graph.get_nodes()}")  # The graph should now have two nodes
print(f"All Edges: {graph.get_edges()}")  # The graph should now have one edge

# Check the data of a node
print("\n---> Checking Node Data")
print(f"Data of node 4: {graph.get_node_data(4)}")  # This should return 'node4'

# Check the weight of an edge
print("\n---> Checking Edge Weight")
print(f"Edge weight between 4 and 5: {graph.get_edge_weight(4, 5)}")  # This should return 6

# Remove a node
graph.remove_node(4)  # Removing node4

print("\n---> After Removing Node 4")
print(f"All Nodes: {graph.get_nodes()}")  # The graph should now have one node

# Check if the graph is empty
print("\n---> Checking If Graph Is Empty")
print(f"Is Graph Empty: {graph.is_empty()}")  # This should return False



---> Initial State of the Graph
All Nodes: []

---> After Adding Nodes
All Nodes: [0, 1, 2, 3]

---> After Adding Edges with Weights
All Edges: [(0, 1, 3), (0, 2, 2), (1, 0, 3), (1, 3, 1), (2, 0, 2), (2, 3, 4), (3, 1, 1), (3, 2, 4)]

---> Edge Existence Check
Edge between 0 and 1: True
Edge between 1 and 0: True

---> Getting Edge Weight
Edge weight between 0 and 1: 3

---> Getting Node Data
Data of node 0: updatedNode0

---> Checking Neighbors of Node 0
Neighbors of Node 0: [1, 2]

---> Checking Graph Properties
Graph Size: 4
Is Graph Empty: False

---> After Removing Some Edges
All Edges: [(0, 2, 2), (1, 3, 1), (2, 0, 2), (3, 1, 1)]

---> After Removing Node 1
All Nodes: [0, 2, 3]

---> After Clearing the Graph
All Nodes: []

---> After Adding Nodes and Edge After Clearing
All Nodes: [4, 5]
All Edges: [(4, 5, 6), (5, 4, 6)]

---> Checking Node Data
Data of node 4: node4

---> Checking Edge Weight
Edge weight between 4 and 5: 6

---> After Removing Node 4
All Nodes: [5]

---> Checkin

#**Graph Search**

**BFS**

In [8]:
def bfs(graph, start_node):
    """
    Perform Breadth-First Search (BFS) on the graph, starting from the node with the specified id.

    Args:
        graph (AdjacencyList): The graph to perform BFS on.
        start_node: The identifier of the starting node.

    Returns:
        paths (dict): A dictionary mapping each node to its shortest path from the source node.
        distances (dict): A dictionary mapping each node to the cost (distance) to reach it from the source node.
    """
    queue = [start_node]
    visited = {start_node}
    paths = {start_node: [start_node]}
    distances = {start_node: 0}

    while queue:
        current_node = queue.pop(0)
        for neighbor in graph.neighbors(current_node):
            if neighbor not in visited:
                queue.append(neighbor)
                visited.add(neighbor)
                paths[neighbor] = paths[current_node] + [neighbor]
                distances[neighbor] = distances[current_node] + 1

    return paths, distances


In [10]:
# Create an instance of AdjacencyList
graph = AdjacencyList(directed=False)

# Add nodes to the graph
for node_id in range(1, 6):
    graph.add_node(node_id)

# Add edges to the graph
edges = [(1, 2), (1, 4), (2, 4), (3, 5), (4, 5)]
for source, destination in edges:
    graph.add_edge(source, destination)

# Perform BFS on the graph from source node 1
paths, distances = bfs(graph, 1)

# Print the path and cost from node 1 to each other node
print("\nBFS Results:\n")
for node_id, path in paths.items():
    cost = distances.get(node_id, 'Infinity')
    print(f"Node {node_id:<2}: Path {str(path):<14}, Cost {cost}")


BFS Results:

Node 1 : Path [1]           , Cost 0
Node 2 : Path [1, 2]        , Cost 1
Node 4 : Path [1, 4]        , Cost 1
Node 5 : Path [1, 4, 5]     , Cost 2
Node 3 : Path [1, 4, 5, 3]  , Cost 3


**DFS**

In [11]:
def dfs(graph, source_id):
    """
    Recursive Depth-First Search (DFS) on the graph.

    Args:
        graph (AdjacencyList): The graph to perform DFS on.
        source_id: The identifier of the current node.

    Returns:
        dict: A dictionary mapping each node to its path, cost, starting and ending time.
              If a node cannot be reached from the source node, its cost will be 'Infinity'.
    """
    paths_costs_times = {node_id: (None, float('Infinity'), None, None) for node_id in graph.nodes}

    if source_id not in graph.nodes:
        return paths_costs_times

    visited = {source_id}
    time = 1
    paths_costs_times[source_id] = ([source_id], 0, time, None)

    stack = [source_id]
    while stack:
        node_id = stack[-1]
        unvisited_neighbors = [neighbor_id for neighbor_id in graph.nodes[node_id].edges if neighbor_id not in visited]

        if unvisited_neighbors:
            next_node_id = unvisited_neighbors[0]
            visited.add(next_node_id)
            time += 1
            paths_costs_times[next_node_id] = (paths_costs_times[node_id][0] + [next_node_id],
                                                paths_costs_times[node_id][1] + 1,
                                                time,
                                                None)
            stack.append(next_node_id)
        else:
            time += 1
            stack.pop()
            if stack:
                paths_costs_times[node_id] = (paths_costs_times[node_id][0],
                                              paths_costs_times[node_id][1],
                                              paths_costs_times[node_id][2],
                                              time)

    return paths_costs_times


In [12]:
# Create an empty graph
graph = AdjacencyList()

# Add nodes to the graph
for node_id in range(1, 6):
    graph.add_node(node_id)

# Add edges to the graph
edges = [(1, 2), (1, 4), (2, 4), (3, 5), (4, 5)]
for source, destination in edges:
    graph.add_edge(source, destination)

# Print the initial state of the graph
print("Initial state of the graph:\n")
print(graph)

# Perform DFS on the graph from source node 1
paths_costs_times = dfs(graph, 1)

# Print the path, cost and start/end time from node 1 to each other node
print("\nDFS Results:\n")
for node_id, (path, cost, start_time, end_time) in paths_costs_times.items():
    cost = 'Infinity' if cost == float('Infinity') else cost
    print(f"Node {node_id:<2}: Path {str(path):<14}, Cost {str(cost):<8}, Start {str(start_time):<5}, End {end_time}")


Initial state of the graph:

<Node(id=1, data=None, edges={2: None, 4: None})>
<Node(id=2, data=None, edges={4: None})>
<Node(id=3, data=None, edges={5: None})>
<Node(id=4, data=None, edges={5: None})>
<Node(id=5, data=None, edges={})>

DFS Results:

Node 1 : Path [1]           , Cost 0       , Start 1    , End None
Node 2 : Path [1, 2]        , Cost 1       , Start 2    , End 7
Node 3 : Path None          , Cost Infinity, Start None , End None
Node 4 : Path [1, 2, 4]     , Cost 2       , Start 3    , End 6
Node 5 : Path [1, 2, 4, 5]  , Cost 3       , Start 4    , End 5


**Topological Sort**

In [13]:
def topological_sort(dfs_result):
    """
    Perform topological sort on a directed acyclic graph.

    Args:
        dfs_result (dict): The result of a depth-first search.

    Returns:
        list: A list of node identifiers in topological order.
    """
    # Sort the nodes by their ending times in descending order
    sorted_nodes = sorted(dfs_result.items(), key=lambda x: x[1][3], reverse=True)

    # Return a list of node identifiers in topological order
    return [node_id for node_id, _ in sorted_nodes]

In [14]:
paths_costs_times

{1: ([1], 0, 1, None),
 2: ([1, 2], 1, 2, 7),
 3: (None, inf, None, None),
 4: ([1, 2, 4], 2, 3, 6),
 5: ([1, 2, 4, 5], 3, 4, 5)}

In [16]:
[[node, finishing_time] for node, (path, cost, start_time, finishing_time) in paths_costs_times.items()]

[[1, None], [2, 7], [3, None], [4, 6], [5, 5]]